# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fae0c26acd0>
├── 'a' --> tensor([[-0.4142, -0.4569, -0.6514],
│                   [ 0.6887, -0.7379,  0.6820]])
└── 'x' --> <FastTreeValue 0x7fae0c26ab50>
    └── 'c' --> tensor([[ 1.6014,  0.9548,  1.1312, -0.2106],
                        [ 2.0915,  0.5527, -1.5771,  1.1464],
                        [-0.2075, -0.6884,  1.1776,  1.8973]])

In [4]:
t.a

tensor([[-0.4142, -0.4569, -0.6514],
        [ 0.6887, -0.7379,  0.6820]])

In [5]:
%timeit t.a

74.3 ns ± 0.906 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fae0c26acd0>
├── 'a' --> tensor([[ 0.2427,  0.0325, -0.0792],
│                   [-0.9434,  0.7082, -0.8126]])
└── 'x' --> <FastTreeValue 0x7fae0c26ab50>
    └── 'c' --> tensor([[ 1.6014,  0.9548,  1.1312, -0.2106],
                        [ 2.0915,  0.5527, -1.5771,  1.1464],
                        [-0.2075, -0.6884,  1.1776,  1.8973]])

In [7]:
%timeit t.a = new_value

84.3 ns ± 0.972 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.4142, -0.4569, -0.6514],
               [ 0.6887, -0.7379,  0.6820]]),
    x: Batch(
           c: tensor([[ 1.6014,  0.9548,  1.1312, -0.2106],
                      [ 2.0915,  0.5527, -1.5771,  1.1464],
                      [-0.2075, -0.6884,  1.1776,  1.8973]]),
       ),
)

In [10]:
b.a

tensor([[-0.4142, -0.4569, -0.6514],
        [ 0.6887, -0.7379,  0.6820]])

In [11]:
%timeit b.a

73.3 ns ± 1.45 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.3250, -0.1579, -0.5066],
               [-0.8283, -1.0801,  1.9358]]),
    x: Batch(
           c: tensor([[ 1.6014,  0.9548,  1.1312, -0.2106],
                      [ 2.0915,  0.5527, -1.5771,  1.1464],
                      [-0.2075, -0.6884,  1.1776,  1.8973]]),
       ),
)

In [13]:
%timeit b.a = new_value

659 ns ± 14.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.11 µs ± 6.77 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

14.3 µs ± 241 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

295 µs ± 5.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

277 µs ± 8.73 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fad66b87ac0>
├── 'a' --> tensor([[[-0.4142, -0.4569, -0.6514],
│                    [ 0.6887, -0.7379,  0.6820]],
│           
│                   [[-0.4142, -0.4569, -0.6514],
│                    [ 0.6887, -0.7379,  0.6820]],
│           
│                   [[-0.4142, -0.4569, -0.6514],
│                    [ 0.6887, -0.7379,  0.6820]],
│           
│                   [[-0.4142, -0.4569, -0.6514],
│                    [ 0.6887, -0.7379,  0.6820]],
│           
│                   [[-0.4142, -0.4569, -0.6514],
│                    [ 0.6887, -0.7379,  0.6820]],
│           
│                   [[-0.4142, -0.4569, -0.6514],
│                    [ 0.6887, -0.7379,  0.6820]],
│           
│                   [[-0.4142, -0.4569, -0.6514],
│                    [ 0.6887, -0.7379,  0.6820]],
│           
│                   [[-0.4142, -0.4569, -0.6514],
│                    [ 0.6887, -0.7379,  0.6820]]])
└── 'x' --> <FastTreeValue 0x7fad66b87d30>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

49.6 µs ± 721 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fad66d98a30>
├── 'a' --> tensor([[-0.4142, -0.4569, -0.6514],
│                   [ 0.6887, -0.7379,  0.6820],
│                   [-0.4142, -0.4569, -0.6514],
│                   [ 0.6887, -0.7379,  0.6820],
│                   [-0.4142, -0.4569, -0.6514],
│                   [ 0.6887, -0.7379,  0.6820],
│                   [-0.4142, -0.4569, -0.6514],
│                   [ 0.6887, -0.7379,  0.6820],
│                   [-0.4142, -0.4569, -0.6514],
│                   [ 0.6887, -0.7379,  0.6820],
│                   [-0.4142, -0.4569, -0.6514],
│                   [ 0.6887, -0.7379,  0.6820],
│                   [-0.4142, -0.4569, -0.6514],
│                   [ 0.6887, -0.7379,  0.6820],
│                   [-0.4142, -0.4569, -0.6514],
│                   [ 0.6887, -0.7379,  0.6820]])
└── 'x' --> <FastTreeValue 0x7fad5fedca30>
    └── 'c' --> tensor([[ 1.6014,  0.9548,  1.1312, -0.2106],
                        [ 2.0915,  0.5527, -1.5771,  1.1464],
                 

In [23]:
%timeit t_cat(trees)

45.5 µs ± 1.03 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

90.1 µs ± 1.34 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.4142, -0.4569, -0.6514],
                [ 0.6887, -0.7379,  0.6820]],
       
               [[-0.4142, -0.4569, -0.6514],
                [ 0.6887, -0.7379,  0.6820]],
       
               [[-0.4142, -0.4569, -0.6514],
                [ 0.6887, -0.7379,  0.6820]],
       
               [[-0.4142, -0.4569, -0.6514],
                [ 0.6887, -0.7379,  0.6820]],
       
               [[-0.4142, -0.4569, -0.6514],
                [ 0.6887, -0.7379,  0.6820]],
       
               [[-0.4142, -0.4569, -0.6514],
                [ 0.6887, -0.7379,  0.6820]],
       
               [[-0.4142, -0.4569, -0.6514],
                [ 0.6887, -0.7379,  0.6820]],
       
               [[-0.4142, -0.4569, -0.6514],
                [ 0.6887, -0.7379,  0.6820]]]),
    x: Batch(
           c: tensor([[[ 1.6014,  0.9548,  1.1312, -0.2106],
                       [ 2.0915,  0.5527, -1.5771,  1.1464],
                       [-0.2075, -0.6884,  1.1776,  1.8973]],
         

In [26]:
%timeit Batch.stack(batches)

108 µs ± 1.98 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.4142, -0.4569, -0.6514],
               [ 0.6887, -0.7379,  0.6820],
               [-0.4142, -0.4569, -0.6514],
               [ 0.6887, -0.7379,  0.6820],
               [-0.4142, -0.4569, -0.6514],
               [ 0.6887, -0.7379,  0.6820],
               [-0.4142, -0.4569, -0.6514],
               [ 0.6887, -0.7379,  0.6820],
               [-0.4142, -0.4569, -0.6514],
               [ 0.6887, -0.7379,  0.6820],
               [-0.4142, -0.4569, -0.6514],
               [ 0.6887, -0.7379,  0.6820],
               [-0.4142, -0.4569, -0.6514],
               [ 0.6887, -0.7379,  0.6820],
               [-0.4142, -0.4569, -0.6514],
               [ 0.6887, -0.7379,  0.6820]]),
    x: Batch(
           c: tensor([[ 1.6014,  0.9548,  1.1312, -0.2106],
                      [ 2.0915,  0.5527, -1.5771,  1.1464],
                      [-0.2075, -0.6884,  1.1776,  1.8973],
                      [ 1.6014,  0.9548,  1.1312, -0.2106],
                      [ 2.0915,  

In [28]:
%timeit Batch.cat(batches)

192 µs ± 1.49 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

562 µs ± 9.95 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
